In [5]:
import numpy as np
%matplotlib inline
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
import uproot
import awkward
import uproot_methods

In [6]:
file = uproot.open("root://cmsxrootd.fnal.gov//store/mc/RunIISummer16NanoAODv4/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano14Dec2018_102X_mcRun2_asymptotic_v6-v1/50000/C9B654CF-0B51-4444-A177-479200DCC429.root")
tree = file["Events"]

In [101]:
tree.keys()

[b'run',
 b'luminosityBlock',
 b'event',
 b'btagWeight_CSVV2',
 b'btagWeight_CMVA',
 b'CaloMET_phi',
 b'CaloMET_pt',
 b'CaloMET_sumEt',
 b'ChsMET_phi',
 b'ChsMET_pt',
 b'ChsMET_sumEt',
 b'nElectron',
 b'Electron_deltaEtaSC',
 b'Electron_dr03EcalRecHitSumEt',
 b'Electron_dr03HcalDepth1TowerSumEt',
 b'Electron_dr03TkSumPt',
 b'Electron_dr03TkSumPtHEEP',
 b'Electron_dxy',
 b'Electron_dxyErr',
 b'Electron_dz',
 b'Electron_dzErr',
 b'Electron_eCorr',
 b'Electron_eInvMinusPInv',
 b'Electron_energyErr',
 b'Electron_eta',
 b'Electron_hoe',
 b'Electron_ip3d',
 b'Electron_jetRelIso',
 b'Electron_mass',
 b'Electron_miniPFRelIso_all',
 b'Electron_miniPFRelIso_chg',
 b'Electron_mvaFall17V1Iso',
 b'Electron_mvaFall17V1noIso',
 b'Electron_mvaFall17V2Iso',
 b'Electron_mvaFall17V2noIso',
 b'Electron_mvaSpring16GP',
 b'Electron_mvaSpring16HZZ',
 b'Electron_pfRelIso03_all',
 b'Electron_pfRelIso03_chg',
 b'Electron_phi',
 b'Electron_pt',
 b'Electron_r9',
 b'Electron_sieie',
 b'Electron_sip3d',
 b'Electron

In [100]:
df = processor.LazyDataFrame(tree)
jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['Jet_pt'].content,
            eta=df['Jet_eta'].content,
            phi=df['Jet_phi'].content,
            mass=df['Jet_mass'].content,
            b_tag=df['Jet_btagDeepB'].content
            )
b_tag = df["Jet_btagDeepB"]

KeyboardInterrupt: 

In [ ]:
trijets = jets.choose(3)

In [ ]:
print(jets.counts.sum())
print((jets.pt > 20).sum().sum())
print((jets.pt > 30).sum().sum())

In [ ]:
def closest(masses):
    delta = abs(172.5 - masses)
    closest_masses = delta.min()
    is_closest = (delta == closest_masses)
    return is_closest

In [ ]:
is_best = trijets[closest(trijets.mass)]

In [ ]:
print(is_best.i0['b_tag'])
print(is_best.i1['b_tag'])
print(is_best.i2['b_tag'])
np.maximum(np.maximum(is_best.i0['b_tag'], is_best.i1['b_tag']), is_best.i2['b_tag'])

In [6]:
df = processor.LazyDataFrame(tree)
muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'].content,
            eta=df['Muon_eta'].content,
            phi=df['Muon_phi'].content,
            mass=df['Muon_mass'].content,
            charge=df['Muon_charge'].content)
electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'].content,
            eta=df['Electron_eta'].content,
            phi=df['Electron_phi'].content,
            mass=df['Electron_mass'].content,
            charge=df['Electron_charge'].content)
MET = df['MET_pt']

In [5]:
three_leptons = (muons.counts + electrons.counts) == 3
muons_3lep = muons[three_leptons]
electrons_3lep = electrons[three_leptons]
eem = electrons_3lep.choose(2).cross(muons_3lep)
mme = muons_3lep.choose(2).cross(electrons_3lep)
eee = electrons_3lep.choose(3)
mmm = muons_3lep.choose(3)

In [19]:
cut = range(0,5)
print(eem[cut])
print()
print(mme[cut])
print()
print(eee[cut])
print()
print(mmm[cut])

[[<tuple 0>] [<tuple 1>] [] [] []]

[[] [] [] [] []]

[[] [] [<tuple 0>] [] [<tuple 1>]]

[[] [] [] [<tuple 0>] []]


In [106]:
good_eem = eem[eem.i0.i0.charge != eem.i0.i1.charge]
good_mme = mme[mme.i0.i0.charge != mme.i0.i1.charge]

# plot transverse mass vs. good_eem.i1.mass
# now just need eee and mmm

''' (1, 2, 3)
    (1, 2) check charge and mass
    (2, 3) check charge and mass
    (3, 1) check charge and mass
    from these three, discard same-charge
    from the remaining, check closest-mass
    graph other lepton

'''

# Takes four hours, oof

def check_leptons(triplets):
    def delta_mass(masses):
        delta = abs(91.2 - masses)
        return delta

    one_two = triplets.i0 + triplets.i1
    two_three = triplets.i1 + triplets.i2
    three_one = triplets.i2 + triplets.i0
    
    opp_12 = (one_two.charge == 0)
    opp_23 = (two_three.charge == 0)
    opp_31 = (three_one.charge == 0)
    
    one_two = one_two[opp_12]
    two_three = two_three[opp_23]
    three_one = three_one[opp_31]
    
    del_12 = np.pad(delta_mass(one_two.p4.mass.content), 1, 'constant', constant_values=1e5)
    del_23 = np.pad(delta_mass(two_three.p4.mass.content), 1, 'constant', constant_values=1e5)
    del_31 = np.pad(delta_mass(three_one.p4.mass.content), 1, 'constant', constant_values=1e5)
    
    third_leptons = np.empty(shape=del_12.shape)
    
    for i in range(del_12.size):
        print(i)
        if (del_12[i] < del_23[i]) & (del_12[i] < del_31[i]):
            np.append(third_leptons, triplets.i2.mass)
        elif (del_23[i] < del_12[i]) & (del_23[i] < del_31[i]):
            np.append(third_leptons, triplets.i0.mass)
        elif (del_31[i] < del_23[i]) & (del_31[i] < del_12[i]):
            np.append(third_leptons, triplets.i1.mass)
    
    return del_12, del_23, del_31, third_leptons

In [6]:
# instead, let's try something else
eee = electrons_3lep[electrons_3lep.counts == 3]
two_eee = eee.choose(2)
cross_eee = two_eee.cross(eee)

In [89]:
mask1 = np.not_equal(cross_eee.i0.i0.p4, cross_eee.i1.p4)
mask2 = np.not_equal(cross_eee.i0.i1.p4, cross_eee.i1.p4)
eee_combos = cross_eee[mask1 & mask2]

In [85]:
import sys
np.set_printoptions(threshold=10)
two_eee.i0[0], two_eee.i1[0], cross_eee.i0.i0[0], cross_eee.i0.i1[0], cross_eee.i1[0], good_eee

(<Table [<Row 4> <Row 4> <Row 5>] at 0x7f8dfbb779b0>,
 <Table [<Row 5> <Row 6> <Row 6>] at 0x7f8dfb6ab080>,
 <Table [<Row 4> <Row 4> <Row 4> ... <Row 5> <Row 5> <Row 5>] at 0x7f8dfbbb6860>,
 <Table [<Row 5> <Row 5> <Row 5> ... <Row 6> <Row 6> <Row 6>] at 0x7f8dfb6abdd8>,
 <Table [<Row 4> <Row 5> <Row 6> ... <Row 4> <Row 5> <Row 6>] at 0x7f8df4450630>,
 <JaggedArrayMethods [[<tuple 2> <tuple 4> <tuple 6>] [<tuple 11> <tuple 13> <tuple 15>] [<tuple 20> <tuple 22> <tuple 24>] ... [<tuple 453674> <tuple 453676> <tuple 453678>] [<tuple 453683> <tuple 453685> <tuple 453687>] [<tuple 453692> <tuple 453694> <tuple 453696>]] at 0x7f8df43ddf98>)

In [102]:
good_eee = eee_combos[eee_combos.i0.i0.charge != eee_combos.i0.i1.charge]
good_eee

def closest_mass(triplets):
    pairs = triplets.i0
    delta = abs(91.2 - pairs.mass)
    closest_masses = delta.min()
    is_closest = (delta == closest_masses)
    return is_closest

best_eee = good_eee[closest_mass(good_eee)]
best_eee

<JaggedArrayMethods [[<tuple 4>] [<tuple 11>] [<tuple 20>] ... [<tuple 453674>] [] [<tuple 453692>]] at 0x7f8df43e2e80>

(50411, 243743)

<JaggedArrayMethods [[] [<tuple 1>] [] ... [] [<tuple 104932>] []] at 0x7f8dfbaa7fd0>

In [ ]:
df = processor.LazyDataFrame(tree)
muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'].content,
            eta=df['Muon_eta'].content,
            phi=df['Muon_phi'].content,
            mass=df['Muon_mass'].content,
            charge=df['Muon_charge'].content
            )
electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'].content,
            eta=df['Electron_eta'].content,
            phi=df['Electron_phi'].content,
            mass=df['Electron_mass'].content,
            charge=df['Electron_charge'].content
            )
MET_pt = df['MET_pt']
MET_phi = df['MET_phi']      
    
def closest(masses):
    delta = abs(91.2 - masses)
    closest_masses = delta.min()
    is_closest = (delta == closest_masses)
    return is_closest
        
def transverse(MET_pts, MET_phis, triplets):
    third_leptons = triplets.i1
    theta = abs(MET_phis - third_leptons.phi)
    massT = 2 * MET_pts * third_leptons.pt * (1 - np.cos(theta))
    return massT
        
three_leptons = (muons.counts + electrons.counts) == 3
muons_3lep = muons[three_leptons]
electrons_3lep = electrons[three_leptons]
        
eem = electrons_3lep.choose(2).cross(muons_3lep)
mme = muons_3lep.choose(2).cross(electrons_3lep)
        
eee = electrons_3lep[electrons_3lep.counts == 3]
eee = eee.choose(2).cross(eee)
eee = eee[np.not_equal(eee.i0.i0.p4, eee.i1.p4) & np.not_equal(eee.i0.i1.p4, eee.i1.p4)]
mmm = muons_3lep[muons_3lep.counts == 3]
mmm = mmm.choose(2).cross(mmm)
mmm = mmm[np.not_equal(mmm.i0.i0.p4, mmm.i1.p4) & np.not_equal(mmm.i0.i1.p4, mmm.i1.p4)]

MET_pt = MET_pt[three_leptons]
MET_phi = MET_phi[three_leptons]

good_eem = eem[eem.i0.i0.charge != eem.i0.i1.charge]
eem_notempty = good_eem.counts > 0
final_eem = good_eem[eem_notempty]
MET_pt_eem = MET_pt[eem_notempty]
MET_phi_eem = MET_phi[eem_notempty]

good_mme = mme[mme.i0.i0.charge != mme.i0.i1.charge]
mme_notempty = good_mme.counts > 0
final_mme = good_mme[mme_notempty]
MET_pt_mme = MET_pt[mme_notempty]
MET_phi_mme = MET_phi[mme_notempty]

MET_pt_3e = MET_pt[electrons_3lep.counts == 3]
MET_phi_3e = MET_phi[electrons_3lep.counts == 3]
MET_pt_3m = MET_pt[muons_3lep.counts == 3]
MET_phi_3m = MET_phi[muons_3lep.counts == 3]

good_eee = eee[eee.i0.i0.charge != eee.i0.i1.charge]
good_mmm = mmm[mmm.i0.i0.charge != mmm.i0.i1.charge]
        
closest_eee = good_eee[closest(good_eee.i0.mass)]
eee_notempty = closest_eee.counts > 0
final_eee = closest_eee[eee_notempty]
MET_pt_eee = MET_pt_3e[eee_notempty]
MET_phi_eee = MET_phi_3e[eee_notempty]

closest_mmm = good_mmm[closest(good_mmm.i0.mass)]
mmm_notempty = closest_mmm.counts > 0
final_mmm = closest_mmm[mmm_notempty]
MET_pt_mmm = MET_pt_3m[mmm_notempty]
MET_phi_mmm = MET_phi_3m[mmm_notempty]

In [206]:
        eem_massT = transverse(MET_pt_eem, MET_phi_eem, final_eem).content
        mme_massT = transverse(MET_pt_mme, MET_phi_mme, final_mme).content
        eee_massT = transverse(MET_pt_eee, MET_phi_eee, final_eee).content
        mmm_massT = transverse(MET_pt_mmm, MET_phi_mmm, final_mmm).content

In [211]:
np.concatenate((eem_massT, mme_massT, eee_massT, mmm_massT)).size, eem_massT.size, mme_massT.size, eee_massT.size, mmm_massT.size

(133171, 54853, 38923, 39395, 0)

In [20]:
electrons

<JaggedArrayMethods [[] [<Row 0> <Row 1>] [] ... [] [<Row 1435559>] [<Row 1435560> <Row 1435561> <Row 1435562>]] at 0x7f06c928dfd0>

In [8]:
def throw(muons, electrons):
    for i in range(muons.size):
        print(i)
        while (muons[i].size + electrons[i].size > 3):
            print(electrons[i])
            print (muons[i].size + electrons[i].size)
            if muons.pt[i].size == 0:
                electrons[i]
            elif electrons.pt[i].size == 0:
                np.delete(muons[i], -1)
                print('2')
            elif muons.pt[i][-1] < electrons[i][-1]:
                np.delete(muons[i], -1)
                print('3')
            else:
                np.delete(electrons[i], 1)
                print('4')

In [9]:
throw(muons, electrons)

0
1
2
3
4
5
6
7
[<Row 7> <Row 8>]
4


/cvmfs/sft.cern.ch/lcg/views/LCG_95apython3/x86_64-slc6-gcc62-opt/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  del sys.path[0]


AttributeError: 'function' object has no attribute 'shape'

In [14]:
muons.pt[muons.counts + electrons.counts > 3], electrons.pt[muons.counts + electrons.counts > 3]

(<JaggedArray [[8.500229 5.5472617] [6.476086] [] ... [23.103392 5.190391] [95.72909] [7.481183]] at 0x7f48127671d0>,
 <JaggedArray [[27.332558 20.614143] [34.66972 25.315931 13.675986 5.7556796] [51.326355 30.170563 13.327804 5.182458] ... [20.299856 15.224225] [43.18431 8.020135 7.155886] [85.57152 22.615715 6.9784274]] at 0x7f48127547b8>)